In [1]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import numpy as np

In [29]:
from qdrant_client import QdrantClient, models

In [2]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [3]:
model = TextEmbedding(model_handle) 

Fetching 5 files: 100%|████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.46it/s]


In [11]:
sentense = 'I just discovered the course. Can I join now?'
embedding = list(model.embed([sentense]))[0]
min_embedding = min(embedding)
print(min_embedding)

-0.11726373885183883


In [13]:
np.linalg.norm(embedding)

np.float64(1.0)

In [15]:
embedding.dot(embedding)

np.float64(1.0000000000000002)

In [16]:
doc = 'Can I still join the course after the start date?'
embeddings = list(model.embed([sentense, doc]))
embeddings[0].dot(embeddings[1])

np.float64(0.9008528895674548)

In [17]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [24]:
answers = []
for doc in documents:
    answers.append(doc['text'])
answers_embedding = np.array(list(model.embed(answers)))
np.array(embedding).dot(answers_embedding.T)

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [26]:
full_texts = []
for doc in documents:
    full_texts.append(doc['question'] + ' ' + doc['text'])
full_text_embedding = np.array(list(model.embed(full_texts)))
np.array(embedding).dot(full_text_embedding.T)

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

In [28]:
model_list = TextEmbedding.list_supported_models()
min_dim = 10000000
for m in model_list:
    if m['dim'] < min_dim:
        min_dim = m['dim']
        min_model = m['model']
print('minimal dimentinality model is: ', min_model, '. the dim is', min_dim)

minimal dimentinality model is:  BAAI/bge-small-en . the dim is 384


## indexing with qdrant

In [54]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [55]:
qd_client = QdrantClient("http://localhost:6333")

In [57]:
collection_name = "zoomcamp-faq"
qd_client.delete_collection(collection_name=collection_name)
# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=min_dim,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [58]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [59]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=min_model)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [60]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [61]:
query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=sentense,
            model=min_model 
        ),
        limit=5,
        with_payload=True
    )

In [62]:
query_points.points[0].score

0.8703172

In [63]:
print(sentense)

I just discovered the course. Can I join now?
